In [2]:
#print("yo yo")
import warnings
warnings.filterwarnings('ignore')


In [3]:
# !pip install ipywidgets;
# !pip install langchain langchain_community langchain_chroma;
# !pip install -qU langchain-groq;
# !pip install -U langchain-huggingface;

In [4]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

 ········


In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(["https://admissions.wisc.edu/apply-as-a-freshman/", "https://admissions.wisc.edu/apply-as-a-transfer/", 
                        "https://admissions.wisc.edu/apply-as-a-reentry-student/", 
                        "https://admissions.wisc.edu/apply-as-a-second-degree-student/"]
)

docs = loader.load()
len(docs[0].page_content)

#docs


21182

In [6]:
len(docs)
print(docs[1].metadata)

{'source': 'https://admissions.wisc.edu/apply-as-a-transfer/', 'title': 'Apply as a Transfer Student – Office of Admissions and Recruitment – UW–Madison', 'description': 'Every year, we are fortunate to receive thousands of\xa0applications\xa0from a diverse range of students who are incredibly bright, engaged, and passionate. They have challenged themselves and those around them to make a difference in the world. They know that the University of Wisconsin–Madison is their next step toward something extraordinary—a place where they can lead.', 'language': 'en-US'}


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=300)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:
# from langchain_core.prompts import PromptTemplate

# template = """You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
# If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
# Use three sentences maximum and keep the answer as concise as possible.
# If the student has not specified what type of student they are, assume First-Year Student. 
# Always say "thanks for asking!" at the end of the answer.

# Context: {context}

# Question: {question}

# Helpful Answer:"""
# college_advisor = PromptTemplate.from_template(template)

In [10]:
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | college_advisor
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is RAG?")

In [11]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
"""You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
Use three sentences maximum and keep the answer as concise as possible.
If the student has not specified what type of student they are, assume First-Year Student. 
Always say "thanks for asking!" at the end of the answer.

Context: {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [12]:
rag_chain.invoke({"input" : "What are the Math requirements for incoming freshman?"}).get('answer', 'No answer found')

'The Math requirement for incoming freshmen is 3 years of math in high school, which must include one year each of algebra, plane geometry, and college-preparatory math. Thanks for asking!'

In [13]:
rag_chain.invoke({"input" : "I missed the early application deadline, what are my options? Can I still reach out to someone?"})

{'input': 'I missed the early application deadline, what are my options? Can I still reach out to someone?',
 'context': [Document(metadata={'description': 'Every year, we are fortunate to receive thousands of\xa0applications\xa0from a diverse range of students who are incredibly bright, engaged, and passionate. They have challenged themselves and those around them to make a difference in the world. They know that the University of Wisconsin–Madison is their next step toward something extraordinary—a place where they can lead.', 'language': 'en-US', 'source': 'https://admissions.wisc.edu/apply-as-a-freshman/', 'title': 'Apply as a First-Year Student – Office of Admissions and Recruitment – UW–Madison'}, page_content='Fall Early Action\nNovember 1\nNovember 8\nOn or before January 31\n\n\nFall Regular Decision\nJanuary 15\nJanuary 22\nOn or before March 31\n\n\n\nApplications and all required application materials must arrive in our office by 11:59 p.m. Pacific Time on the noted deadlin

In [17]:
#%pip install langgraph

In [21]:
# RAG Agent approach:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()

# Build retriever tool
tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]


agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [26]:
from langchain_core.messages import HumanMessage, SystemMessage

config = {"configurable": {"thread_id": "abc123"}}

query = "What are the Math requirements for incoming freshman?"
for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_th9c', 'function': {'arguments': '{"query":"Math requirements for incoming freshman"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 929, 'total_tokens': 979, 'completion_time': 0.160007407, 'prompt_time': 0.068597862, 'queue_time': 0.012744027000000005, 'total_time': 0.228605269}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0699d263-547a-4c92-b694-654cae29d981-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Math requirements for incoming freshman'}, 'id': 'call_th9c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 50, 'total_tokens': 979})]}}
----
{'tools': {'messages': [ToolMessage(content='Subject Area\nMinimum Requirement \n\n\n\n\nEnglish\n4 years\n\n\nMath *\n3 years\n\n